In [1]:
 !pip install google-generativeai pandas tqdm openpyxl

In [4]:

import pandas as pd
from tqdm import tqdm
import time
import google.generativeai as genai

file_path = "/content/dataset_input.xlsx"   
all_sheets = pd.read_excel(file_path, sheet_name=None)  # dict: {sheet_name: DataFrame}

genai.configure(api_key="-----------")  
model = genai.GenerativeModel("models/gemini-2.0-flash-lite")

# Define tone styles
all_user_styles = {
    "Neutral": """
You are an assistant that writes neutral and professional invitation emails.
Keep the tone formal, clear, and polite, avoiding emotional or overly friendly language.
""",
    "Casual_Friendly_User": """
You are a person who writes friendly and polite invitations.
Keep it conversational, warm, and respectful — a perfect mix of casual and professional tone.
Use contractions naturally (like “we're”, “you’ll”) and sound approachable.
""",
    "Formal_Authoritative_User": """
You are a senior professional or manager sending an official invitation.
Keep the tone highly formal, precise, and authoritative — no unnecessary warmth.
""",
    "Simple_Email_Tone": """
Write a simple and direct email in plain format like this:

Subject: Meeting held
Hello Team,
Meeting scheduled tomorrow, everyone must come.
Venue: meeting room
Time: 4 pm
Date: 12 October 2025,
etc ...
Regards,
Manager

Follow that pattern strictly and fill in the given event details properly.
"""
}

sheet_user_combinations = {
    0: ["Neutral", "Casual_Friendly_User"],        # Sheet 1
    1: ["Neutral", "Formal_Authoritative_User"],   # Sheet 2
    2: ["Neutral", "Simple_Email_Tone"],           # Sheet 3
}

# =============================================
#  Generate and store responses
# =============================================
print("🧠 Generating email invitations using Google Gemini...\n")

final_sheets = {}

for sheet_index, (sheet_name, event_data) in enumerate(all_sheets.items()):
    print(f"\n==============================")
    print(f"📄 Processing Sheet {sheet_index+1}: {sheet_name}")
    print("==============================")

    selected_users = sheet_user_combinations.get(sheet_index, ["Neutral"])
    responses_by_user = {user: [] for user in selected_users}

    # Loop through each event
    for idx, row in tqdm(event_data.iterrows(), total=len(event_data)):
        date = row.get('date', 'N/A')
        time_ = row.get('Time', 'N/A')
        venue = row.get('Venue', 'N/A')
        occasion = row.get('OccasionType', 'N/A')
        host = row.get('Host', 'N/A')
        event = row.get('Event', 'N/A')

        for user in selected_users:
            style_instruction = all_user_styles[user]
            prompt = f"""
{style_instruction}

Write an email inviting participants to the following event.
Ensure the email tone matches the style instruction and stays under 100 words if not using the simple format.

Event Details:
- Date: {date}
- Time: {time_}
- Venue: {venue}
- Occasion Type: {occasion}
- Host: {host}
- Event: {event}
"""

            try:
                response = model.generate_content(prompt)
                cleaned = response.text.strip()
            except Exception as e:
                cleaned = f"[Error generating response: {e}]"

            responses_by_user[user].append(cleaned)

            # Sleep to respect rate limits (adjust delay if you hit errors)
            time.sleep(2)

    # Add generated responses as new columns in the DataFrame
    for user in selected_users:
        event_data[f"response_{user}"] = responses_by_user[user]

    final_sheets[sheet_name] = event_data

print("\n✅ Generation complete! Now saving all sheets to one Excel file...")

output_path = "/content/ir_pro4_with_responses3.xlsx"

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    for sheet_name, df in final_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"\n🎉 All done! File saved successfully:\n📁 {output_path}")


🧠 Generating email invitations using Google Gemini...


📄 Processing Sheet 1: Sheet1


100%|██████████| 20/20 [02:21<00:00,  7.06s/it]



📄 Processing Sheet 2: Sheet2


100%|██████████| 20/20 [02:22<00:00,  7.13s/it]



📄 Processing Sheet 3: Sheet3


100%|██████████| 20/20 [02:14<00:00,  6.73s/it]


✅ Generation complete! Now saving all sheets to one Excel file...

🎉 All done! File saved successfully:
📁 /content/ir_pro4_with_responses3.xlsx
